In [1]:
%matplotlib inline
import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')


AntsPy version = 0.5.4
SimpleITK version = 2.5.0


In [2]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\CHEEZYJEEZY\Desktop\MRIdata_preprocessing


In [3]:
raw_examples = [
    'fsl-open-dev_sub-001_T1w.nii.gz',
    'wash-120_sub-001_T1w.nii.gz',
    'kf-panda_sub-01_ses-3T_T1w.nii.gz',
    'listen-task_sub-UTS01_ses-1_T1w.nii.gz'
]

raw_example = raw_examples[3]

In [4]:
raw_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w.nii.gz')
raw_img_ants = ants.image_read(raw_img_path, reorient = 'IAL')

explore_3D_array(raw_img_ants.numpy(), cmap = 'nipy_spectral')

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Template based method (Native Space)

Template image

In [5]:
template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii' )
template_img_ants = ants.image_read(template_img_path, reorient = 'IAL')

explore_3D_array(template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [6]:
mask_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w_mask.nii.gz')
mask_img_ants = ants.image_read(mask_img_path, reorient = 'IAL')

explore_3D_array(mask_img_ants.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [7]:
np.unique(mask_img_ants.numpy())

array([0.000000e+00, 3.773869e-03, 6.423873e-03, ..., 2.549955e+02,
       2.549986e+02, 2.550000e+02], dtype=float32)

Register template to raw image

In [8]:
transformation = ants.registration(
    fixed = raw_img_ants,
    moving = template_img_ants,
    type_of_transform = 'SyN',
    verbose = True
)

antsRegistration -d 3 -r [00000242B91D1068,00000242B91D0A28,1] -m mattes[00000242B91D1068,00000242B91D0A28,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[00000242B91D1068,00000242B91D0A28,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\CHEEZY~1\AppData\Local\Temp\tmpamf8caz5,00000242B91D1088,00000242B91D1EE8] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [9]:
print(transformation)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz51Warp.nii.gz', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz50GenericAffine.mat'], 'invtransforms': ['C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz50GenericAffine.mat', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz51InverseWarp.nii.gz']}


In [10]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array_comparison(
    arr_before = raw_img_ants.numpy(),
    arr_after = registered_img_ants.numpy()
)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [12]:
mask_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w_mask.nii.gz')
mask_template_img_ants = ants.image_read(mask_img_path, reorient = 'IAL')


Apply the generated transformations to the mask of template

In [14]:
brain_mask = ants.apply_transforms(
    fixed = transformation['warpedmovout'],
    moving = mask_template_img_ants,
    transformlist = transformation['fwdtransforms'],
    interpolator = 'nearestNeighbor',
    verbose = True
)

['-d', '3', '-i', '00000242C1CDEBE8', '-o', '00000242B94A3D28', '-r', '00000242C1CDEEC8', '-n', 'nearestNeighbor', '-t', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz51Warp.nii.gz', '-t', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpamf8caz50GenericAffine.mat']


In [ ]:
explore_3D_array( brain_mask.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…